In [3]:
import pandas as pd
from datetime import datetime,timedelta

In [2]:
# Open the file and read it line by line
file_path = "annual1.txt"
data_list = []  # List to hold dictionaries

with open(file_path, "r") as file:
    # Read the first line, strip quotes from headers, and split by commas
    header = [col.strip('"') for col in file.readline().strip().split(",")]
    
    # Process each subsequent line
    for line in file:
        # Split the line by commas and strip quotes from each value
        values = [value.strip('"') for value in line.strip().split(",")]
        
        # Create a dictionary for the row
        row_dict = dict(zip(header, values))
        
        # Append the dictionary to the list
        data_list.append(row_dict)
    
df=pd.DataFrame(data_list)

columns=['Market and Exchange Names',
'As of Date in Form YYYY-MM-DD','Open Interest (All)',
'Noncommercial Positions-Long (All)',
'Noncommercial Positions-Short (All)',
'Commercial Positions-Long (All)',
'Commercial Positions-Short (All)',
' Total Reportable Positions-Long (All)',
'Total Reportable Positions-Short (All)',
'Nonreportable Positions-Long (All)',
'Nonreportable Positions-Short (All)',
'Change in Open Interest (All)',
'Change in Noncommercial-Long (All)',
'Change in Noncommercial-Short (All)',
'Change in Noncommercial-Spreading (All)',
'Change in Commercial-Long (All)',
'Change in Commercial-Short (All)',
'Change in Total Reportable-Long (All)',
'Change in Total Reportable-Short (All)',
'Change in Nonreportable-Long (All)',
'Change in Nonreportable-Short (All)']
df=df[columns]
df=df.reset_index(drop=True)

df['As of Date in Form YYYY-MM-DD'] = pd.to_datetime(df['As of Date in Form YYYY-MM-DD'], dayfirst=True, errors='coerce')
for i in df.columns[2:]:
    # print(i)
    df[i] = pd.to_numeric(df[i], errors='coerce').astype('Int64') 

# col_dict={
#     'Noncommercial':['Noncommercial Positions-Long (All)','Noncommercial Positions-Short (All)'],
#     'Commercial':['Commercial Positions-Long (All)','Commercial Positions-Short (All)'],
#     'Total Reportable':[' Total Reportable Positions-Long (All)','Total Reportable Positions-Short (All)'],
#     'Nonreportable':['Nonreportable Positions-Long (All)','Nonreportable Positions-Short (All)']}

col_dict={
    'Noncommercial':['Change in Noncommercial-Long (All)','Change in Noncommercial-Short (All)'],
    'Commercial':['Change in Commercial-Long (All)', 'Change in Commercial-Short (All)']}

for a, b in col_dict.items():
    df[f'{a}_net_positions'] = df[b[0]] - df[b[1]]
df.fillna(0,inplace=True)

def generate_market_signals(df):
    """
    Analyzes market positions of Non-Commercials (speculators) and Commercials (hedgers)
    to generate trading signals.

    Parameters:
    df (DataFrame): DataFrame containing columns:
        - 'Market and Exchange Names'
        - 'As of Date in Form YYYY-MM-DD'
        - 'Noncommercial_net_positions'
        - 'Commercial_net_positions'

    Returns:
    DataFrame: Original DataFrame with an additional 'Market_Signal' column
    indicating 'Bullish', 'Bearish', or 'Neutral' signal.
    """
    def determine_signal(non_com, com):
        if non_com > 0 and com < 0:
            return 'Bullish'
        elif non_com < 0 and com > 0:
            return 'Bearish'
        else:
            return 'Neutral'
    
    # Apply the signal determination function
    df['Market_Signal'] = df.apply(
        lambda row: determine_signal(
            row['Noncommercial_net_positions'], 
            row['Commercial_net_positions']
        ), axis=1
    )
    
    return df

updated_df = generate_market_signals(df)

# Display the DataFrame with signals
# print(updated_df[['Market and Exchange Names', 'As of Date in Form YYYY-MM-DD', 'Market_Signal']])

updated_df=updated_df[['Market and Exchange Names', 'As of Date in Form YYYY-MM-DD','Noncommercial_net_positions','Commercial_net_positions','Market_Signal']]

updated_df['As of Date in Form YYYY-MM-DD']=pd.to_datetime(updated_df['As of Date in Form YYYY-MM-DD'])

updated_df=updated_df[updated_df['As of Date in Form YYYY-MM-DD']>datetime.now()-timedelta(days=15)]

target_strings = [
    'DOLLAR', 'SILVER', 'GOLD', 'POUND', 'FRANC', 'USD INDEX', 'EURO FX', 
    'JAPANESE YEN', 'DOW JONES', 'BITCOIN', 'S&P 500', 'ETHER', 'DOGECOIN', 
    'NASDAQ', 'NIKKEI', 'OIL'
]

filtered_df = updated_df[updated_df.iloc[:, 0].apply(lambda x: any(target_string in str(x) for target_string in target_strings))]

filtered_df=filtered_df.reset_index(drop=True).sort_values(by='Noncommercial_net_positions')


C:\Users\user\AppData\Local\Temp\ipykernel_5340\457647230.py:45: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['As of Date in Form YYYY-MM-DD'] = pd.to_datetime(df['As of Date in Form YYYY-MM-DD'], dayfirst=True, errors='coerce')
C:\Users\user\AppData\Local\Temp\ipykernel_5340\457647230.py:62: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.fillna(0,inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_5340\457647230.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

In [3]:
updated_df

,Market and Exchange Names,As of Date in Form YYYY-MM-DD,Noncommercial_net_positions,Commercial_net_positions,Market_Signal
0,WHEAT-SRW - CHICAGO BOARD OF TRADE,2024-11-19,-7510,6614,Bearish
47,WHEAT-HRW - CHICAGO BOARD OF TRADE,2024-11-19,560,-2991,Bullish
94,WHEAT-HRSpring - MINNEAPOLIS GRAIN EXCHANGE,2024-11-19,-6200,5892,Bearish
141,CORN - CHICAGO BOARD OF TRADE,2024-11-19,-14345,12969,Bearish
188,OATS - CHICAGO BOARD OF TRADE,2024-11-19,-305,116,Bearish
...,...,...,...,...,...
14554,S&P 500 ANNUAL DIVIDEND INDEX - CHICAGO MERCAN...,2024-11-19,-336,89,Bearish
14601,S&P 500 QUARTERLY DIVIDEND IND - CHICAGO MERCA...,2024-11-19,-202,202,Bearish
14637,GULF JET NY HEAT OIL SPR - NEW YORK MERCANTILE...,2024-11-19,150,-150,Bullish
14684,MARINE .5% FOB USGC/BRENT 1st - ICE FUTURES EN...,2024-11-19,13,-13,Bullish


In [4]:
#---------------------------------------------------------------------------------------

In [13]:
file_path = "annual1.txt"
data_list = []  # List to hold dictionaries

with open(file_path, "r") as file:
    # Read the first line, strip quotes from headers, and split by commas
    header = [col.strip('"') for col in file.readline().strip().split(",")]
    
    # Process each subsequent line
    for line in file:
        # Split the line by commas and strip quotes from each value
        values = [value.strip('"') for value in line.strip().split(",")]
        
        # Create a dictionary for the row
        row_dict = dict(zip(header, values))
        
        # Append the dictionary to the list
        data_list.append(row_dict)
    
df=pd.DataFrame(data_list)

columns=['Market and Exchange Names',
'As of Date in Form YYYY-MM-DD','Open Interest (All)',
'Noncommercial Positions-Long (All)',
'Noncommercial Positions-Short (All)',
'Commercial Positions-Long (All)',
'Commercial Positions-Short (All)',
' Total Reportable Positions-Long (All)',
'Total Reportable Positions-Short (All)',
'Nonreportable Positions-Long (All)',
'Nonreportable Positions-Short (All)',
'Change in Open Interest (All)',
'Change in Noncommercial-Long (All)',
'Change in Noncommercial-Short (All)',
'Change in Noncommercial-Spreading (All)',
'Change in Commercial-Long (All)',
'Change in Commercial-Short (All)',
'Change in Total Reportable-Long (All)',
'Change in Total Reportable-Short (All)',
'Change in Nonreportable-Long (All)',
'Change in Nonreportable-Short (All)']
df=df[columns]
df=df.reset_index(drop=True)

df['As of Date in Form YYYY-MM-DD'] = pd.to_datetime(df['As of Date in Form YYYY-MM-DD'], dayfirst=True, errors='coerce')
for i in df.columns[2:]:
    # print(i)
    df[i] = pd.to_numeric(df[i], errors='coerce').astype('Int64') 

# col_dict={
#     'Noncommercial':['Noncommercial Positions-Long (All)','Noncommercial Positions-Short (All)'],
#     'Commercial':['Commercial Positions-Long (All)','Commercial Positions-Short (All)'],
#     'Total Reportable':[' Total Reportable Positions-Long (All)','Total Reportable Positions-Short (All)'],
#     'Nonreportable':['Nonreportable Positions-Long (All)','Nonreportable Positions-Short (All)']}

col_dict={
    'Noncommercial':['Change in Noncommercial-Long (All)','Change in Noncommercial-Short (All)'],
    'Commercial':['Change in Commercial-Long (All)', 'Change in Commercial-Short (All)']}

for a, b in col_dict.items():
    df[f'{a}_net_positions'] = df[b[0]] - df[b[1]]
df.fillna(0,inplace=True)

df['As of Date in Form YYYY-MM-DD']=pd.to_datetime(df['As of Date in Form YYYY-MM-DD'])

C:\Users\user\AppData\Local\Temp\ipykernel_8932\1622252650.py:44: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['As of Date in Form YYYY-MM-DD'] = pd.to_datetime(df['As of Date in Form YYYY-MM-DD'], dayfirst=True, errors='coerce')
C:\Users\user\AppData\Local\Temp\ipykernel_8932\1622252650.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.fillna(0,inplace=True)


In [14]:
df=df[['Market and Exchange Names', 'As of Date in Form YYYY-MM-DD','Noncommercial_net_positions','Commercial_net_positions']].sort_values(by='As of Date in Form YYYY-MM-DD')


In [7]:
df=df[df['As of Date in Form YYYY-MM-DD']>datetime.now()-timedelta(days=60)]	

In [9]:
target_strings = [
    'DOLLAR', 'SILVER', 'GOLD', 'POUND', 'FRANC', 'USD INDEX', 'EURO FX', 
    'JAPANESE YEN', 'DOW JONES', 'BITCOIN', 'S&P 500', 'ETHER', 'DOGECOIN', 
    'NASDAQ', 'NIKKEI', 'OIL'
]

df = df[df.iloc[:, 0].apply(lambda x: any(target_string in str(x) for target_string in target_strings))]

In [24]:
df=df.sort_values(by=['Market and Exchange Names','As of Date in Form YYYY-MM-DD']).reset_index(drop=True)

In [29]:
df.iloc[:,0].value_counts()

Market and Exchange Names
ADJUSTED INT RATE S&P 500 TOTL - CHICAGO MERCANTILE EXCHANGE    7
NZ DOLLAR - CHICAGO MERCANTILE EXCHANGE                         7
MICRO ETHER - CHICAGO MERCANTILE EXCHANGE                       7
MICRO GOLD - COMMODITY EXCHANGE INC.                            7
NANO ETHER - LMX LABS LLC                                       7
NASDAQ MINI - CHICAGO MERCANTILE EXCHANGE                       7
NASDAQ-100 Consolidated - CHICAGO MERCANTILE EXCHANGE           7
NIKKEI STOCK AVERAGE YEN DENOM - CHICAGO MERCANTILE EXCHANGE    7
S&P 500 ANNUAL DIVIDEND INDEX - CHICAGO MERCANTILE EXCHANGE     7
AUSTRALIAN DOLLAR - CHICAGO MERCANTILE EXCHANGE                 7
S&P 500 Consolidated - CHICAGO MERCANTILE EXCHANGE              7
S&P 500 QUARTERLY DIVIDEND IND - CHICAGO MERCANTILE EXCHANGE    7
SILVER - COMMODITY EXCHANGE INC.                                7
SOYBEAN OIL - CHICAGO BOARD OF TRADE                            7
SWISS FRANC - CHICAGO MERCANTILE EXCHANGE         

In [7]:
df.to_excel('cot.xlsx')

In [12]:
df[df['Market and Exchange Names']=='CANADIAN DOLLAR - CHICAGO MERCANTILE EXCHANGE']

,Market and Exchange Names,As of Date in Form YYYY-MM-DD,Noncommercial_net_positions,Commercial_net_positions
11849,CANADIAN DOLLAR - CHICAGO MERCANTILE EXCHANGE,2024-01-02,13590,-15628
11848,CANADIAN DOLLAR - CHICAGO MERCANTILE EXCHANGE,2024-01-09,13751,-10107
11847,CANADIAN DOLLAR - CHICAGO MERCANTILE EXCHANGE,2024-01-16,-6008,6715
11846,CANADIAN DOLLAR - CHICAGO MERCANTILE EXCHANGE,2024-01-23,4937,-1767
11845,CANADIAN DOLLAR - CHICAGO MERCANTILE EXCHANGE,2024-01-30,6063,-7361
11844,CANADIAN DOLLAR - CHICAGO MERCANTILE EXCHANGE,2024-02-06,-5348,8612
11843,CANADIAN DOLLAR - CHICAGO MERCANTILE EXCHANGE,2024-02-13,2254,2546
11842,CANADIAN DOLLAR - CHICAGO MERCANTILE EXCHANGE,2024-02-20,4619,-5613
11841,CANADIAN DOLLAR - CHICAGO MERCANTILE EXCHANGE,2024-02-27,-515,-1918
11840,CANADIAN DOLLAR - CHICAGO MERCANTILE EXCHANGE,2024-03-05,-18459,24140
